In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
import cv2
import os

In [ ]:
img = image.load_img("./data/archiveshort/validation/3lci8x.jpg")
plt.imshow(img)

In [ ]:
cv2.imread("./data/archiveshort/validation/3lci8x.jpg").shape

In [ ]:
path_treino = "./data/archiveshort/data/train/"
path_test = "./data/archiveshort/data/test/"

In [ ]:
# Pré-processamento dos dados
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
        path_treino,
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

validation_dataset = test_datagen.flow_from_directory(
        path_test,
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

In [ ]:
train_dataset.class_indices

In [ ]:
import json

data = train_dataset.class_indices

with open('./model/dataset_indices.json', 'w') as f:
    json.dump(data, f)

In [ ]:
model = load_model("./model/modelo_treinado.h5")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Realizar a validação cruzada
# Criar o classificador Keras
classifier = KerasClassifier(build_fn=model, epochs=10, batch_size=32, verbose=0)

# Realizar a validação cruzada
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

X_train, y_train = next(train_dataset)
X_test, y_test = next(validation_dataset)

scores = cross_val_score(classifier, X_train, y_train, cv=kfold, scoring='accuracy')

# Imprimir os scores
# print("Scores:", scores)
# print("Accuracy:", scores.mean())

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict


predictions = cross_val_predict(classifier, X_train, y_train, cv=4)
report = classification_report(y_train, predictions)

In [ ]:
print(report)
print("Accuracy:", scores.mean())

In [ ]:
plt.plot(model.history["loss"])
plt.plot(model.history["val_loss"])
plt.title("Histórico de Treinamento")
plt.ylabel("Função de Custo")
plt.xlabel("Épocas de treinamento")
plt.legend(["Erro treino", "Erro teste"])
plt.show()

In [ ]:
from PIL import Image

dire = os.getcwd()

diretorio = dire + "/data/archiveshort/validation_oficial/"
# diretorio = dire + "/data/archiveshort/data/validation/"

print(diretorio)

for filename in os.listdir(diretorio):
  

      if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):

        img_path = os.path.join(diretorio, filename)
      
        img = Image.open(img_path)
        # imagem_redimensionada = img.resize((100,100))
        # imagem_redimensionada.save(img_path)        
        
        # img = image.load_img(img_path)
        img = img.resize((100,100))
        plt.imshow(img)
        plt.show()
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])
        value = model.predict(images)
        value = value[0].tolist()
        
        value_list_int = list(map(int, value))         
        
        maxValue = max(value_list_int)        
        valu_posi = value_list_int.index(maxValue)
        
        
        dictClassifier = train_dataset.class_indices
        
        for i,each in enumerate(dictClassifier):
          
          if(valu_posi == i):
            print(each)